In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\

In [2]:
file_path = 'stats_df.csv'
stats_df = pd.read_csv(
    file_path,
    index_col=None,
    infer_datetime_format=True,
    parse_dates=True,
)
stats_df.head()

FileNotFoundError: [Errno 2] File b'stats_df.csv' does not exist: b'stats_df.csv'

In [ ]:
# Drop player with 0 OPS
stats_df = stats_df[stats_df.OPS !=0].set_index('Player')

In [ ]:
stats_df.head()

In [ ]:
# Creating X & y variables
X = stats_df.iloc[:, 4:-2]
y = stats_df["OPS"].values

In [ ]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train.shape

In [ ]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

y = y.reshape(-1, 1)
scaler_y = StandardScaler().fit(y)
y = scaler_y.transform(y)

In [ ]:
y_train.shape

In [ ]:
# Model set-up
number_input_features = 17
hidden_nodes_layer1 = 34
hidden_nodes_layer2 = 5

In [ ]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

In [ ]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [ ]:
# Training the model
batch_size = 1000
epochs = 100
training_history = model.fit(
    X_train,
    y_train,
    #validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

In [ ]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)

# Evaluate the model
model.evaluate(X_test, y_test)

In [ ]:
predicted_OPS = scaler_y.inverse_transform(predicted)

In [ ]:
moneyball = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": predicted_OPS.ravel()
    }, index = stats_df.index[-len(predicted_OPS): ])
moneyball.head(20)

In [ ]:
moneyball = moneyball.groupby('Player').mean()
moneyball.head(20)

In [ ]:
moneyball.plot(figsize=(12,8))